# Finding & Enriching Small & Medium Sized Businesses with Sixtyfour

This notebook teaches you on how to use the Sixtyfour API to find and enrich businesses via our api. To get started, go to https://app.sixtyfour.ai and sign up to create an account with an API key.

### Set up

In [68]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import asyncio
import httpx

In [71]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
load_dotenv()

True

In [11]:
SIXTYFOUR_API_KEY = os.getenv('SIXTYFOUR_API_KEY')

### Searching for businesses

In [38]:
search_query = 'Find me all photgraphers in La Jolla, California'

Feel free to setup any fields of your choice, the search agent might not have all the information on these in one go, but we will enrich in the next step

In [39]:
fields={
    "company_name":"Name of the company",
    "company_email":"Email for the company",
    "address":"Where are they located",
    "city":"City they are located in",
    "state":"Which state do they operate in",
    "phone_number":"Their phone number",
    "company_linkedin":"Linkedin profile of the company if available",
    "google_profile":"Google business profile if found",
    "website":"Website of the company",
    "key_people":"name of people here seperated by commas"
  }

This step takes a bit as the agent is searching the internet for businesses!

In [40]:
res = requests.post('https://api.sixtyfour.ai/search', headers={
    'x-api-key': SIXTYFOUR_API_KEY,
    "Content-Type": "application/json"
}, json={
    "search_query": search_query,
    "struct": fields
})

In [41]:
df = pd.DataFrame(res.json())

In [43]:
df.head()

,company_name,company_email,address,city,state,phone_number,company_linkedin,google_profile,website,key_people,notes,extra_emails
0,Maria V Photography,,"13634 Vian Rd, Poway, CA 92064",Poway,CA,+18133352197,,,https://www.mariavphotography.com/,,"Photographer. Identifies as women-owned, offer...",
1,Zeena Gregg Photography,,"15604 Riparian Rd, Poway, CA 92064",Poway,CA,+18587800510,,https://www.google.com/maps/place/data=!3m1!4b...,http://www.zeenagregg.com/,,Lead identified as a photographer with a 5-sta...,
2,FarPix,,,,,+18584052023,,,https://www.farpix.com/,,FarPix is a photography service with additiona...,
3,IDM Photos,,"11579 Poblado Rd #b, San Diego, CA 92127",San Diego,CA,+18584927377,,,http://www.idmphotos.com/,,Business is a wedding and portrait photographe...,
4,Fotility Studios,,,,,+18004941405,,https://www.google.com/maps/place/data=!3m1!4b...,http://www.fotilitystudios.com/,,Google listing indicates 24-hour operation eve...,


In [44]:
df.shape

(265, 12)

### Next step is to enrich these companies on the account level

Let's first enrich one company and then we will run this on the dataframe async!

In [48]:
random_company = df.iloc[15].to_dict()
random_company

{'company_name': 'Elle G Photography',
 'company_email': '',
 'address': '1070 Sapphire St, San Diego, CA 92109',
 'city': 'San Diego',
 'state': 'CA',
 'phone_number': '+18583441068',
 'company_linkedin': '',
 'google_profile': '',
 'website': 'https://ellegphotography.com/',
 'key_people': '',
 'notes': 'Business is women-owned and offers online appointments. Appointment required. Rated 5 with 37 reviews.',
 'extra_emails': ''}

In [55]:
# We can add some new columns in account level enrichment if we want! (It will auto contain the original columns)
new_fields = {
    "instagram_url":'Instagram url for the photography company',
    "num_employees":"How many employees work there, give approximation if you don't have exact number"
}

In [52]:
# Find People Toggle --> looks for people at the company
find_people = True
people_focus_prompt="Find me the owners of the company and the office manager"
lead_struct = {
    "name":"name of the person",
    "title":"title of the person at the company",
    "email":"email of the person",
    "phone_number":"phone number of the person",
    "instagram_url":"instagram url for the person"
    ## Add more fields optionally
}

In [56]:
# Optionally, we can also add a research plan to better guide the agent for specifics
research_plan = "Check their website, online profiles, and linkedin for the people. Looking for the individual portfolios of the employees can help too"

In [60]:
req = requests.post(
    "https://api.sixtyfour.ai/enrich-company",
    headers={
        'x-api-key': SIXTYFOUR_API_KEY,
        "Content-Type": "application/json"
    },
    json = {
        "target_company":random_company,
        "struct":new_fields,
        "find_people":find_people,
        "research_plan":research_plan,
        "people_focus_prompt":people_focus_prompt
    }
)

In [62]:
res = req.json()

In [63]:
res.keys()

dict_keys(['notes', 'structured_data', 'findings'])

In [64]:
res['structured_data']

{'instagram_url': 'https://www.instagram.com/ellegphotography/',
 'num_employees': 'Less than 10 core staff members',
 'company_name': 'Elle G Photography',
 'company_email': '',
 'address': '1070 Sapphire St, San Diego, CA 92109',
 'city': 'San Diego',
 'state': 'CA',
 'phone_number': '+18583441068',
 'company_linkedin': '',
 'google_profile': '',
 'website': 'https://ellegphotography.com/',
 'key_people': 'Owner and Principal Photographer: Lisa Gisczinski (Lisa G.)',
 'notes': 'Business is women-owned and offers online appointments. Appointment required. Rated 5 with 37 reviews.',
 'extra_emails': '',
 'leads': [{'name': 'Lisa Gisczinski',
   'email': 'ellegphotography@me.com',
   'company': 'Elle G Photography',
   'title': 'Owner and Principal Photographer',
   'phone': '+18583441068',
   'score': '9',
   'linkedin': 'https://www.linkedin.com/in/studioellephotography'}]}

In [65]:
res['notes']

"Elle G Photography is a women-owned boutique photography business located at 1070 Sapphire St, San Diego, CA 92109, specializing in beach, family, engagement, maternity, and senior photography with services by appointment only. Their contact number is +1 858-344-1068, and the official website is https://ellegphotography.com/, which features their portfolio and photography style. The company maintains a high reputation with a 5-star rating based on 37 reviews. The business operates with fewer than 10 core employees, confirming its boutique nature despite some third-party claims of a larger staff. The Instagram profile https://www.instagram.com/ellegphotography/ is active, showcasing their work and company culture. The founder and principal photographer is Lisa Gisczinski, also known as Lisa G., confirmed via her LinkedIn profile at https://www.linkedin.com/in/studioellephotography, which identifies her as the owner and leader. There is no publicly available, verified information about 

In [66]:
res['findings']

['Owner identified as Lisa Gisczinski (Lisa G.), principal photographer and founder of Elle G Photography.',
 'Company Instagram is https://www.instagram.com/ellegphotography/, personal account of Elle G with modest followers (4) and 29 posts.',
 'Phone number is +1 858-344-1068 matching the business listing.',
 'Business is women-owned with appointment-only service model and 5-star rating based on 37 reviews.',
 'Employee count reported variably as approx. 25 per ZoomInfo but likely a small boutique team possibly fewer than 10 core members.',
 'No specific office manager identified from available public information, no professional LinkedIn company page available (dead link).',
 "Owner's LinkedIn profile found: https://www.linkedin.com/in/studioellephotography, listing freelance photographer and business owner at Elle G Photography.",
 'Company: Elle G Photography, located at 1070 Sapphire St, San Diego, CA 92109',
 'Owner and Principal Photographer: Lisa Gisczinski (Lisa G.)',
 "Lisa

The agent not only filled out most of the information but also made notes (feel free to ignore if not useful)!

### Running this on the entire dataframe async!

In [69]:
async def enrich_company(
    client: httpx.AsyncClient,
    semaphore: asyncio.Semaphore,
    company: dict,
    max_retries: int = 3
):
    """Enrich company info using the SixtyFour API with retries and concurrency limits."""
    async with semaphore:
        for attempt in range(max_retries):
            try:
                response = await client.post(
                    "https://api.sixtyfour.ai/enrich-company",
                    headers={
                        'x-api-key': SIXTYFOUR_API_KEY,
                        "Content-Type": "application/json"
                    },
                    json={
                        "target_company": company,
                        "struct": new_fields,
                        "find_people": find_people,
                        "research_plan": research_plan,
                        "people_focus_prompt": people_focus_prompt
                    }
                )
                response.raise_for_status()
                print(f"Response JSON: {str(response.text)[:20]}")
                return response.json()
            except httpx.HTTPStatusError as http_err:
                print(f"HTTP error occurred: {http_err}\nResponse: {response.text}")
            except httpx.RequestError as req_err:
                print(f"Request error occurred: {req_err}")
                if attempt < max_retries - 1:
                    print(f"Retrying ({attempt + 1}/{max_retries})...")
                    await asyncio.sleep(2 ** attempt)
                    continue
            except Exception as e:
                print(f"Unexpected error: {e}")
            return None
    return None

async def process_leads(df: pd.DataFrame) -> list:
    results = []
    semaphore = asyncio.Semaphore(100)
    timeout = httpx.Timeout(300.0, connect=10.0)

    async with httpx.AsyncClient(follow_redirects=True, timeout=timeout) as client:
        tasks = [enrich_company(client, semaphore, df.iloc[i].to_dict()) for i in range(len(df))]
        results = await asyncio.gather(*tasks, return_exceptions=True)

    return results

I am only running on the first 20, but feel free to just pass in df instead of df.head(20) to run it on the full dataframe.

results = asyncio.run(process_leads(df))

In [72]:
results = asyncio.run(process_leads(df.head(20)))

Response JSON: {"notes":"Tayler Ash
Response JSON: {"notes":"FarPix is 
Response JSON: {"notes":"IDM Photos
Response JSON: {"notes":"James Aron
Response JSON: {"notes":"Zeena Greg
Response JSON: {"notes":"Maria V Ph
Response JSON: {"notes":"Elle G Pho
Response JSON: {"notes":"Jen Jordan
Response JSON: {"notes":"Madinalmph
Response JSON: {"notes":"Lisa Engla
Response JSON: {"notes":"Christy Wa
Response JSON: {"notes":"La Jolla D
Response JSON: {"notes":"Jess Livin
Response JSON: {"notes":"Reimo Phot
Response JSON: {"notes":"Shadi Amer
Response JSON: {"notes":"Fotility S
Response JSON: {"notes":"ABM Weddin
Response JSON: {"notes":"Savoring T
Response JSON: {"notes":"La Jolla a
Request error occurred: 
Retrying (1/3)...
Response JSON: {"notes":"Brant Bend


In [76]:
df_enriched = pd.DataFrame(results)
df_enriched

,notes,structured_data,findings
0,Maria V Photography is a woman-owned family ph...,{'instagram_url': 'https://www.instagram.com/m...,"[Owner identified as Maria Velasquez, owner an..."
1,Zeena Gregg Photography is a small photography...,{'instagram_url': 'https://www.instagram.com/z...,[Owner: Zeena Gregg. Identified as Owner/Photo...
2,"FarPix is a professional photography, graphic ...","{'instagram_url': '', 'num_employees': 'Fewer ...",[FarPix is a 5-star rated professional photogr...
3,IDM Photos is a well-established wedding and p...,{'instagram_url': 'https://www.instagram.com/d...,"[IDM Photos is located at 11579 Poblado Rd #b,..."
4,Fotility Studios is a full-service photography...,{'instagram_url': 'https://www.instagram.com/f...,[Fotility Studios operates 24/7 with a 4.7 rat...
5,Savoring The Sweet Life Photography is a bouti...,{'instagram_url': 'https://www.instagram.com/s...,[Pam Davis is the owner and primary photograph...
6,ABM Wedding Photography is a highly rated wedd...,{'instagram_url': 'https://www.instagram.com/a...,"[Owner: Brian Kent, Office Manager: Courtney (..."
7,Brant Bender Photography operates from 13330 P...,{'instagram_url': 'https://www.instagram.com/b...,"[Company Name: Brant Bender Photography, Busin..."
8,Lisa England Photography is a women-owned bout...,{'instagram_url': 'https://www.instagram.com/l...,[Lisa England Photography is a women-owned pho...
9,Tayler Ashley Photography is a Southern Califo...,{'instagram_url': 'https://www.instagram.com/t...,[Tayler Ashley Photography is based in Souther...


In [80]:
df_enriched_data = pd.DataFrame(df_enriched['structured_data'].to_list())
df_enriched_data

,instagram_url,num_employees,company_name,company_email,address,city,state,phone_number,company_linkedin,google_profile,website,key_people,notes,extra_emails,leads
0,https://www.instagram.com/maria.vphotography/,1 (likely solo operation),Maria V Photography,,"13634 Vian Rd, Poway, CA 92064",Poway,CA,+1 813-335-2197,,,https://www.mariavphotography.com/,Maria Velasquez,"Photographer. Identifies as women-owned, offer...",,"[{'name': 'Maria Velasquez', 'email': '', 'com..."
1,https://www.instagram.com/zeenagreggphotography/,0-9,Zeena Gregg Photography,,"15604 Riparian Rd, Poway, CA 92064",Poway,CA,+18587800510,,https://www.google.com/maps/place/data=!3m1!4b...,http://www.zeenagregg.com/,,Lead identified as a photographer with a 5-sta...,,"[{'name': 'Zeena Gregg', 'email': '', 'company..."
2,,Fewer than 10,FarPix,,,San Diego / Orange County,California,+18584052023,,,https://www.farpix.com/,Owner: Amir Farhad Bakhshi,FarPix is a photography service with additiona...,,"[{'name': 'Amir Farhad Bakhshi', 'email': '', ..."
3,https://www.instagram.com/dbgibby_idmphotos/,1,IDM Photos,idmphotos2@gmail.com,"11579 Poblado Rd #b, San Diego, CA 92127",San Diego,CA,+18584927377,,,http://www.idmphotos.com/,David Gibby - Owner and Principal Photographer,Business is a wedding and portrait photographe...,,"[{'name': 'David Gibby', 'email': 'idmphotos2@..."
4,https://www.instagram.com/fotility/,2-10,Fotility Studios,,8029 Westmore Road,San Diego,California,+18004941405,https://www.linkedin.com/company/fotility,https://www.google.com/maps/place/data=!3m1!4b...,http://www.fotilitystudios.com/,"Jerry Roxas (Owner/Operator, Chief Creative Of...",Google listing indicates 24-hour operation eve...,,"[{'name': 'Jerry Roxas', 'email': '', 'company..."
5,https://www.instagram.com/savoringthesweetlife/,"Fewer than 25, primarily Pam Davis",Savoring The Sweet Life Photography,,,San Diego,California,,,https://www.google.com/maps/place/data=!3m1!4b...,http://www.savoringthesweetlife.com/,Pam Davis (Owner and Principal Photographer),Photography service operating 24 hours daily w...,,"[{'name': 'Pam Davis', 'email': '', 'company':..."
6,https://www.instagram.com/abmphotographysd/,Approximately 3,ABM Wedding Photography,info@abmfoto.com,"3421 Tripp Ct STE 2, San Diego, CA 92121",San Diego,CA,+19494633038,,,https://www.abmweddingphotos.com/,"Brian Kent (Owner), Courtney (Office Manager /...",This lead is a wedding photographer with a hig...,,"[{'name': 'Brian Kent', 'email': 'info@abmfoto..."
7,https://www.instagram.com/brantbenderphotography/,Approximately 1 (sole proprietor),Brant Bender Photography,,"13330 Paseo Del Verano Norte, Poway, CA 92064",Poway,CA,+18582184082,,,http://www.brantbender.com/,Brant Bender,Category: Wedding photographer. 24/7 operating...,,"[{'name': 'Brant Bender', 'email': '', 'compan..."
8,https://www.instagram.com/lisaenglandphotography/,fewer than 25,Lisa England Photography,Lisa@lisaenglandphotography.com,12186 Via Milano,San Diego,CA,+18583959987,,,http://www.lisaenglandphotography.com/,Lisa England,Photographer business identified as women-owne...,,"[{'name': 'Lisa England', 'email': 'Lisa@lisae..."
9,https://www.instagram.com/taylerashleyphotogra...,1,Tayler Ashley Photography,info@taylerashley-photo.com,,,Southern California,,,https://www.google.com/maps/place/data=!3m1!4b...,http://www.taylerashley-photo.com/,Tayler Suterko (Owner and Photographer),This is a photography service with a 4.9 ratin...,,"[{'name': 'Tayler Suterko', 'email': 'info@tay..."


This is our account level results, feel free to save this in a csv and export it like this:

In [81]:
# df_enriched_data.to_csv('account_level.csv')

To extract individual leads from this, we can use the next function

In [82]:
def generate_lead_rows(company_data):
    """
    Given a company_data dictionary containing company info and a list of leads,
    this function returns a list of dictionaries, where each dictionary represents
    a row containing both the company information and the individual lead details.
    """
    company_info = {key: value for key, value in company_data.items() if key != "leads"}
    lead_rows = []

    for lead in company_data.get("leads", []):
        row = company_info.copy()
        row.update(lead)
        lead_rows.append(row)

    if not lead_rows:
        return [company_info]

    return lead_rows

In [87]:
individual_leads = pd.DataFrame(df_enriched_data.apply(generate_lead_rows,axis=1).sum())
individual_leads

,instagram_url,num_employees,company_name,company_email,address,city,state,phone_number,company_linkedin,google_profile,...,key_people,notes,extra_emails,name,email,company,title,phone,score,linkedin
0,https://www.instagram.com/maria.vphotography/,1 (likely solo operation),Maria V Photography,,"13634 Vian Rd, Poway, CA 92064",Poway,CA,+1 813-335-2197,,,...,Maria Velasquez,"Photographer. Identifies as women-owned, offer...",,Maria Velasquez,,Maria V Photography,"Owner, Primary Photographer, Business Coach, E...",+1 813-335-2197,9,https://www.linkedin.com/in/maria-velasquez-81...
1,https://www.instagram.com/zeenagreggphotography/,0-9,Zeena Gregg Photography,,"15604 Riparian Rd, Poway, CA 92064",Poway,CA,+18587800510,,https://www.google.com/maps/place/data=!3m1!4b...,...,,Lead identified as a photographer with a 5-sta...,,Zeena Gregg,,Zeena Gregg Photography,Owner / Lead Photographer,+18587800510,8,
2,,Fewer than 10,FarPix,,,San Diego / Orange County,California,+18584052023,,,...,Owner: Amir Farhad Bakhshi,FarPix is a photography service with additiona...,,Amir Farhad Bakhshi,,FarPix,Owner,,8,
3,https://www.instagram.com/dbgibby_idmphotos/,1,IDM Photos,idmphotos2@gmail.com,"11579 Poblado Rd #b, San Diego, CA 92127",San Diego,CA,+18584927377,,,...,David Gibby - Owner and Principal Photographer,Business is a wedding and portrait photographe...,,David Gibby,idmphotos2@gmail.com,IDM Photos,Owner and Principal Photographer,+1 858-492-7377,10,https://www.linkedin.com/in/david-gibby-79693553
4,https://www.instagram.com/fotility/,2-10,Fotility Studios,,8029 Westmore Road,San Diego,California,+18004941405,https://www.linkedin.com/company/fotility,https://www.google.com/maps/place/data=!3m1!4b...,...,"Jerry Roxas (Owner/Operator, Chief Creative Of...",Google listing indicates 24-hour operation eve...,,Jerry Roxas,,Fotility Studios,"Owner / Operator, Chief Creative Officer",,9,https://www.linkedin.com/in/jerryroxas
5,https://www.instagram.com/savoringthesweetlife/,"Fewer than 25, primarily Pam Davis",Savoring The Sweet Life Photography,,,San Diego,California,,,https://www.google.com/maps/place/data=!3m1!4b...,...,Pam Davis (Owner and Principal Photographer),Photography service operating 24 hours daily w...,,Pam Davis,,Savoring The Sweet Life Photography,Owner and Principal Photographer,,9,https://www.linkedin.com/in/pam-davis-15a9b023
6,https://www.instagram.com/abmphotographysd/,Approximately 3,ABM Wedding Photography,info@abmfoto.com,"3421 Tripp Ct STE 2, San Diego, CA 92121",San Diego,CA,+19494633038,,,...,"Brian Kent (Owner), Courtney (Office Manager /...",This lead is a wedding photographer with a hig...,,Brian Kent,info@abmfoto.com,ABM Wedding Photography,Owner,+1 949-463-3038,9,https://www.linkedin.com/in/abmfoto
7,https://www.instagram.com/abmphotographysd/,Approximately 3,ABM Wedding Photography,info@abmfoto.com,"3421 Tripp Ct STE 2, San Diego, CA 92121",San Diego,CA,+19494633038,,,...,"Brian Kent (Owner), Courtney (Office Manager /...",This lead is a wedding photographer with a hig...,,Courtney,,ABM Wedding Photography,Office Manager / Assistant Photographer,,6,
8,https://www.instagram.com/brantbenderphotography/,Approximately 1 (sole proprietor),Brant Bender Photography,,"13330 Paseo Del Verano Norte, Poway, CA 92064",Poway,CA,+18582184082,,,...,Brant Bender,Category: Wedding photographer. 24/7 operating...,,Brant Bender,,Brant Bender Photography,"Owner, Chief Photographer & Creative Director",+18582184082,9,https://www.linkedin.com/in/brantbender
9,https://www.instagram.com/lisaenglandphotography/,fewer than 25,Lisa England Photography,Lisa@lisaenglandphotography.com,12186 Via Milano,San Diego,CA,+18583959987,,,...,Lisa England,Photographer business identified as women-owne...,,Lisa England,Lisa@lisaenglandphotography.com,Lisa England Photography,Owner/Photographer,+1 858-395-9987,9,https://www.linkedin.com/in/lisa-england-16818838


We can also save these individual accounts, but first, one good way to going about this is to find the emails here so we can run email campaigns!

In [91]:
async def fetch_email(client: httpx.AsyncClient, semaphore: asyncio.Semaphore, lead: dict, max_retries: int = 3):
    """Fetch email with concurrency control and retry mechanism."""
    async with semaphore:
        for attempt in range(max_retries):
            try:
                response = await client.post("https://api.sixtyfour.ai/find-email",
                 headers={
                        'x-api-key': SIXTYFOUR_API_KEY,
                        "Content-Type": "application/json"
                    },
                json={"lead": lead})
                response.raise_for_status()
                print(f"Response JSON: {str(response.text)[:20]}")
                return response.text
            except httpx.HTTPStatusError as http_err:
                print(f"HTTP error occurred: {http_err}\nResponse: {response.text}")
            except httpx.RequestError as req_err:
                print(f"Request error occurred: {req_err}")
                if attempt < max_retries - 1:
                    print(f"Retrying ({attempt + 1}/{max_retries})...")
                    await asyncio.sleep(2 ** attempt)  # Exponential backoff
                    continue
            except Exception as e:
                print(f"Unexpected error: {e}")
            return None
    return None

async def process_leads_email(df: pd.DataFrame) -> list:
    results = []
    semaphore = asyncio.Semaphore(100)  # Limit to 100 concurrent requests
    timeout = httpx.Timeout(300.0, connect=10.0)
    
    async with httpx.AsyncClient(follow_redirects=True, timeout=timeout) as client:
        tasks = [fetch_email(client, semaphore, df.iloc[i].to_dict()) for i in range(len(df))]
        results = await asyncio.gather(*tasks, return_exceptions=True)
    
    return results

In [129]:
results_email = asyncio.run(process_leads_email(individual_leads))

Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"",
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"",
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht
Response JSON: {"instagram_url":"ht


In [130]:
final_df = pd.DataFrame([eval(r) for r in results_email])

The returned emails include metadata to assist with identifying potential bounces. It’s generally a good idea to clean out any emails marked as “unknown,” since their deliverability couldn’t be verified. However, if you’re okay with taking on a bit of risk, these “unknown” emails might still be worth using in some cases.

In [134]:
final_df.iloc[7]['email']

'claunchie@gmail.com'

In [132]:
final_df['email'] = final_df['email'].apply(
    lambda x: x if not isinstance(x, list) else x[0][0] if x and x[0][1] != 'UNKNOWN' else ''
)

Here's how many emails we where able to find!

In [133]:
final_df[['name','company','title','email','phone','score']]

,name,company,title,email,phone,score
0,Maria Velasquez,Maria V Photography,"Owner, Primary Photographer, Business Coach, E...",,+1 813-335-2197,9
1,Zeena Gregg,Zeena Gregg Photography,Owner / Lead Photographer,zeena@zeenagregg.com,+18587800510,8
2,Amir Farhad Bakhshi,FarPix,Owner,,,8
3,David Gibby,IDM Photos,Owner and Principal Photographer,idmphotos2@gmail.com,+1 858-492-7377,10
4,Jerry Roxas,Fotility Studios,"Owner / Operator, Chief Creative Officer",,,9
5,Pam Davis,Savoring The Sweet Life Photography,Owner and Principal Photographer,,,9
6,Brian Kent,ABM Wedding Photography,Owner,info@abmfoto.com,+1 949-463-3038,9
7,Courtney,ABM Wedding Photography,Office Manager / Assistant Photographer,claunchie@gmail.com,,6
8,Brant Bender,Brant Bender Photography,"Owner, Chief Photographer & Creative Director",,+18582184082,9
9,Lisa England,Lisa England Photography,Owner/Photographer,Lisa@lisaenglandphotography.com,+1 858-395-9987,9


In [136]:
final_df.head()

,instagram_url,num_employees,company_name,company_email,address,city,state,phone_number,company_linkedin,google_profile,...,key_people,notes,extra_emails,name,email,company,title,phone,score,linkedin
0,https://www.instagram.com/maria.vphotography/,1 (likely solo operation),Maria V Photography,,"13634 Vian Rd, Poway, CA 92064",Poway,CA,+1 813-335-2197,,,...,Maria Velasquez,"Photographer. Identifies as women-owned, offer...",,Maria Velasquez,,Maria V Photography,"Owner, Primary Photographer, Business Coach, E...",+1 813-335-2197,9,https://www.linkedin.com/in/maria-velasquez-81...
1,https://www.instagram.com/zeenagreggphotography/,0-9,Zeena Gregg Photography,,"15604 Riparian Rd, Poway, CA 92064",Poway,CA,+18587800510,,https://www.google.com/maps/place/data=!3m1!4b...,...,,Lead identified as a photographer with a 5-sta...,,Zeena Gregg,zeena@zeenagregg.com,Zeena Gregg Photography,Owner / Lead Photographer,+18587800510,8,
2,,Fewer than 10,FarPix,,,San Diego / Orange County,California,+18584052023,,,...,Owner: Amir Farhad Bakhshi,FarPix is a photography service with additiona...,,Amir Farhad Bakhshi,,FarPix,Owner,,8,
3,https://www.instagram.com/dbgibby_idmphotos/,1,IDM Photos,idmphotos2@gmail.com,"11579 Poblado Rd #b, San Diego, CA 92127",San Diego,CA,+18584927377,,,...,David Gibby - Owner and Principal Photographer,Business is a wedding and portrait photographe...,,David Gibby,idmphotos2@gmail.com,IDM Photos,Owner and Principal Photographer,+1 858-492-7377,10,https://www.linkedin.com/in/david-gibby-79693553
4,https://www.instagram.com/fotility/,2-10,Fotility Studios,,8029 Westmore Road,San Diego,California,+18004941405,https://www.linkedin.com/company/fotility,https://www.google.com/maps/place/data=!3m1!4b...,...,"Jerry Roxas (Owner/Operator, Chief Creative Of...",Google listing indicates 24-hour operation eve...,,Jerry Roxas,,Fotility Studios,"Owner / Operator, Chief Creative Officer",,9,https://www.linkedin.com/in/jerryroxas


In [138]:
# Now we can save this data and import it into salesforce/instantly/some other CRM!!
#final_df.to_csv('leads.csv')